This is a demo of how to use 'rok_estimator.py'.

In [1]:
from rok_estimator.rok_estimator import *

In [2]:
# The example below shows how to simulate the executions of individual RoKs.

# Specify parameters related to the underlying ring
ring_60 = RingParam(f=60)

# Create a new starting relation
rel = Relation(ring=ring_60,wdim=60,log_beta_wit_inf=0)
rel.show()

# Running pi_bdecomp reducing rel to rel_bdecomp
rel_bdecomp = rel.pi_bdecomp(ell=2)
rel_bdecomp.show()

# Running pi_split reducing rel into rel_split
rel_split = rel.pi_split(2)
rel_split.show()

# Running pi_batch reducing rel into rel_batch
rel_batch = rel.pi_batch()
rel_batch.show()

# Running pi_fold reducing rel into rel_fold
rel_fold = rel.pi_fold(23)
rel_fold.show()

# Running pi_norm reducing rel into rel_norm
rel_norm = rel.pi_norm()
rel_norm.show()

 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |       60 |   1 |        (  8 |  0 )        |         (  0 |  0 )         | 120.0 B  |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 bdecomp   |       60 |   2 |        (  8 |  0 )        |         (  0 |  0 )         | 240.0 B  |      (5.000 KB | 5.000 KB)      |         (2^-oo  | 2^-oo )         
 
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 split     |       30 |   2 |        (  8 |  0 )        |         (  0 |  0 )         | 120.0 B  |      (5.000 KB | 5.000 KB)      |         (2^-128 | 2^-12

In [3]:
# The example below shows how to simulate the execution of a sequence of RoKs with specified parameters.

# Parameters for the underlying ring
ring_params = {
    "f": 60,
    "log_beta_sis_2": 40,
    "log_q": 64,
}

# Parameters for the starting relation
rel_params = {
    "wdim": 2**22,
    "rep": 25,
    "log_beta_wit_inf": 0,
}

# Parameters for the RoKs
ops_params = {
    "ell": 4,
    "d": 2,
}

# The sequence of RoKs
opener = [("norm", {}), ("batch", {}), ("split", {"d": ops_params["d"]}), ("fold", {})]
loop = [("bdecomp", {"ell": ops_params["ell"]}), ("norm", {}), ("batch", {}), ("split", {"d": ops_params["d"]}), ("fold", {})]
ops = opener + 10 * loop + [("finish", {})]

# Simulate the execution of the sequence of RoKs
sim = Simulation(ring_params, rel_params)
sim.execute(ops)
sim.extract()
sim.show()

Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  4194304 |  25 |        ( 16 | 16 )        |         (  0 | 16 )         | 200.0 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 norm      |  4194304 |  42 |        ( 16 | 33 )        |         (  0 | 30 )         | 336.0 MB |      (145.4 KB | 145.4 KB)      |         (2^-123 | 2^-123)         
 batch     |  4194304 |  42 |        ( 16 | 33 )        |         (  0 | 30 )         | 336.0 MB |      (145.4 KB | 145.4 KB)      |         (2^-121 | 2^-120)         
 split     |  2097152 |  84 |        ( 16 | 33 )        |         (  0 | 30 )         | 336.0 MB |      (336.0 KB | 481.4 KB)      |         (2^-128 | 2^-120)         
 fold      |  2097152 |  25 |        ( 23 | 25 )        |         (  9 | 25 )         | 1000. MB |      (336.0 KB | 481.4 KB)      |         (2

In [4]:
# The example below shows how to define new RoKs and use them in simulation. 

class NewRelation(Relation):
    def execute(self, op, **kwargs):
        match op:
            case "new":
                return self.pi_new()
            case _:
                return super().execute(op, **kwargs)
            
    def pi_new(self):
        comm = 0 
        snd_err = 0 
        rel_params = {
            # "ring": self.ring,
            # "trivial": self.trivial,
            # "n_compress": self.n_compress,
            # "n_commit": self.n_commit,
            # "n_rel": self.n_rel,
            # "wdim": self.wdim,
            # "rep": self.rep,
            # "log_beta_wit_2": bound_log_canon_2_from_log_coeff_inf(self.ring,self.log_beta_wit_inf, dim=self.wdim * (self.rep + ell)), # Measured in Frobenius norm
            # "log_beta_wit_2": self.log_beta_wit_2, # Measured in max ell_2-norm over all columns
            # "log_beta_wit_inf": self.log_beta_wit_inf
            "comm" : comm,      
            "acc_comm" : self.acc_comm + comm,       
            "snd_err" : snd_err,
            "acc_snd_err" : self.acc_snd_err + snd_err,           
            # "log_beta_ext_2_func" : lambda x : x, 
            # "log_beta_ext_inf_func" : lambda x : x, 
        }      
        return replace(self, **rel_params)
    
ring_params = {
    "f": 60,
    "log_beta_sis_2": 36,
    "log_q": 64,
}

rel_params = {
    "wdim": 2**16 * 41,
    "rep": 25,
    "log_beta_wit_inf": 0,
}

ops = [("new", {}), ("finish", {})]

sim = Simulation(ring_params, rel_params, NewRelation)
sim.execute(ops)
sim.extract()
sim.show()

Execution Trace:
 operation |   wdim   | rep | log_2-norm  (real | extr) | log_inf-norm  (real | extr) | wit size | communication  (growth | total) | soundness error  (growth | total) 
 init      |  2686976 |  25 |        ( 16 | 16 )        |         (  0 |  0 )         | 128.1 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 init      |  2686976 |  25 |        ( 16 | 16 )        |         (  0 |  0 )         | 128.1 MB |      (0.0000 B | 0.0000 B)      |         (2^-oo  | 2^-oo )         
 finish    |          |     |                           |                             |          |      (128.1 MB | 128.1 MB)      |         (2^-oo  | 2^-oo )         
 
Total Cost: communication = 128.1 MB, soundness error = 2^-oo
Maximum log ell_2-norm (real | extr) = ( 16 | 16 ), log SIS norm bound = 36
 
